# Aave V2 Wallet Credit Scoring in Google Colab

### This notebook processes Aave V2 transaction data to generate credit scores (0-1000) for each wallet based on their historical behavior. Higher scores indicate responsible usage, while lower scores indicate risky or bot-like activity.



In [35]:
# Step 1: Install required packages
!pip install pandas numpy lightgbm matplotlib pyarrow

# Step 2: Import libraries
import os
import json
import pandas as pd
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt
from lightgbm import LGBMRegressor
from google.colab import drive

In [36]:
# Step 3: Mount Google Drive (where your data is stored)
# drive.mount('/content/drive')

import zipfile
import os

# Step 4: Define the path to your uploaded .zip file
file_path = 'user-wallet-transactions.json.zip'  # the name must match what you uploaded

# Step 5: Unzip the file
with zipfile.ZipFile(file_path, 'r') as zip_ref:
    zip_ref.extractall('/content/')  # extract to /content/

# Optional: List extracted files
extracted_files = os.listdir('/content/')
print("Extracted files:", extracted_files)


Extracted files: ['.config', 'user-wallet-transactions.json.zip', 'score_distribution.png', '__MACOSX', 'user-wallet-transactions.json', 'sample_data']


In [37]:
import json

json_path = '/content/user-wallet-transactions.json'
with open(json_path, 'r') as f:
    data = json.load(f)

# Preview
print(data[:2])  # or use pandas if it's a list of dicts


[{'_id': {'$oid': '681d38fed63812d4655f571a'}, 'userWallet': '0x00000000001accfa9cef68cf5371a23025b6d4b6', 'network': 'polygon', 'protocol': 'aave_v2', 'txHash': '0x695c69acf608fbf5d38e48ca5535e118cc213a89e3d6d2e66e6b0e3b2e8d4190', 'logId': '0x695c69acf608fbf5d38e48ca5535e118cc213a89e3d6d2e66e6b0e3b2e8d4190_Deposit', 'timestamp': 1629178166, 'blockNumber': 1629178166, 'action': 'deposit', 'actionData': {'type': 'Deposit', 'amount': '2000000000', 'assetSymbol': 'USDC', 'assetPriceUSD': '0.9938318274296357543568636362026045', 'poolId': '0x2791bca1f2de4661ed88a30c99a7a9449aa84174', 'userId': '0x00000000001accfa9cef68cf5371a23025b6d4b6'}, '__v': 0, 'createdAt': {'$date': '2025-05-08T23:06:39.465Z'}, 'updatedAt': {'$date': '2025-05-08T23:06:39.465Z'}}, {'_id': {'$oid': '681aa70dd6df53021cc6f3c0'}, 'userWallet': '0x000000000051d07a4fb3bd10121a343d85818da6', 'network': 'polygon', 'protocol': 'aave_v2', 'txHash': '0xe6fc162c86b2928b0ba9b82bda672763665152b9de9d92b0e1512a81b1129e3f', 'logId': '0

In [38]:
# Step 5: Data Loading Function
def load_data(file_path):
    # Check if file exists
    if not os.path.exists(file_path):
        raise FileNotFoundError(f"File not found at {file_path}")

    # Check if file is zip
    if file_path.endswith('.zip'):
        import zipfile
        with zipfile.ZipFile(file_path, 'r') as zip_ref:
            # Extract the json file (assuming it's the only file)
            json_file_path = None
            for file in zip_ref.namelist():
                if file.endswith('.json'):
                    zip_ref.extract(file)
                    json_file_path = file
                    break
            if json_file_path is None:
                raise FileNotFoundError(f"No JSON file found in the zip archive at {file_path}")
            file_path = json_file_path


    print(f"Loading data from {file_path}...")

    # Load JSON data
    with open(file_path, 'r') as f:
        data = json.load(f)

    print("Creating DataFrame...")
    df = pd.DataFrame(data)
    print(f"Loaded {len(df)} transactions")

    return df

In [39]:
# Step 6: Feature Engineering
def engineer_features(df):
    print("Engineering features...")

    # Convert timestamp to datetime
    df['timestamp'] = pd.to_datetime(df['timestamp'], unit='s')

    # Group by wallet address
    grouped = df.groupby('userWallet')

    features = []

    for wallet, transactions in grouped:
        wallet_features = {'userWallet': wallet}

        # Basic counts
        wallet_features['n_transactions'] = len(transactions)
        wallet_features['n_deposits'] = (transactions['action'] == 'deposit').sum()
        wallet_features['n_borrows'] = (transactions['action'] == 'borrow').sum()
        wallet_features['n_repays'] = (transactions['action'] == 'repay').sum()
        wallet_features['n_liquidations'] = (transactions['action'] == 'liquidationcall').sum()

        # Time-based features
        transactions = transactions.sort_values('timestamp')
        time_diff = transactions['timestamp'].diff().dt.total_seconds()
        wallet_features['avg_time_between_tx'] = time_diff.mean()
        wallet_features['time_since_first_tx'] = (datetime.now() - transactions['timestamp'].min()).days

        # Amount-based features
        # Extract amount and convert to numeric, handling potential errors
        transactions['actionData_amount'] = transactions['actionData'].apply(lambda x: x.get('amount', '0'))
        transactions['actionData_amount'] = pd.to_numeric(transactions['actionData_amount'], errors='coerce').fillna(0)

        # Extract assetPriceUSD and convert to numeric
        transactions['actionData_assetPriceUSD'] = transactions['actionData'].apply(lambda x: x.get('assetPriceUSD', '0'))
        transactions['actionData_assetPriceUSD'] = pd.to_numeric(transactions['actionData_assetPriceUSD'], errors='coerce').fillna(0)

        # Calculate amountUSD
        transactions['amountUSD'] = transactions['actionData_amount'] * transactions['actionData_assetPriceUSD']

        if 'amountUSD' in transactions.columns:
            deposits = transactions.loc[transactions['action'] == 'deposit', 'amountUSD']
            borrows = transactions.loc[transactions['action'] == 'borrow', 'amountUSD']

            wallet_features['total_deposits'] = deposits.sum()
            wallet_features['total_borrows'] = borrows.sum()
            wallet_features['avg_deposit'] = deposits.mean()
            wallet_features['avg_borrow'] = borrows.mean()
            wallet_features['deposit_borrow_ratio'] = wallet_features['total_deposits'] / (wallet_features['total_borrows'] + 1e-6)

        # Risk flags
        wallet_features['has_liquidation'] = wallet_features['n_liquidations'] > 0
        wallet_features['high_frequency'] = wallet_features['avg_time_between_tx'] is not None and wallet_features['avg_time_between_tx'] < 10  # seconds
        wallet_features['new_account'] = wallet_features['time_since_first_tx'] < 7  # < 1 week

        features.append(wallet_features)

    features_df = pd.DataFrame(features)

    # Fill NA values
    features_df.fillna({
        'total_deposits': 0,
        'total_borrows': 0,
        'avg_deposit': 0,
        'avg_borrow': 0,
        'deposit_borrow_ratio': 0,
        'avg_time_between_tx': 0
    }, inplace=True)

    print(f"Engineered features for {len(features_df)} wallets")
    return features_df

In [40]:
# Step 7: Rule-Based Scoring
def rule_based_scoring(wallets):
    print("Applying rule-based scoring...")

    scores = []

    for _, wallet in wallets.iterrows():
        # Rule 1: Liquidations (very bad)
        if wallet['has_liquidation']:
            scores.append(200)
            continue

        # Rule 2: High frequency (potential bot)
        if wallet['high_frequency']:
            scores.append(100)
            continue

        # Rule 3: Very new account with borrowing
        if wallet['new_account'] and wallet['n_borrows'] > 0:
            scores.append(300)
            continue

        # Rule 4: No deposits but borrowing (risky)
        if wallet['n_deposits'] == 0 and wallet['n_borrows'] > 0:
            scores.append(250)
            continue

        # Passed rule-based checks
        scores.append(None)

    wallets['rule_based_score'] = scores
    return wallets


In [41]:
# Step 8: Machine Learning Scoring
def ml_scoring(wallets):
    print("Applying machine learning scoring...")

    # Filter wallets that need ML scoring
    ml_wallets = wallets[wallets['rule_based_score'].isna()].copy()

    if len(ml_wallets) == 0:
        print("No wallets for ML scoring")
        wallets['ml_score'] = np.nan
        return wallets

    # Features for ML model
    features = [
        'n_transactions', 'n_deposits', 'n_borrows', 'n_repays',
        'avg_time_between_tx', 'time_since_first_tx',
        'total_deposits', 'total_borrows', 'deposit_borrow_ratio'
    ]

    # Create target variable (simulated - in practice you'd need real labels)
    # Higher scores for wallets with good deposit/borrow ratios and long history
    ml_wallets['target'] = (
        (ml_wallets['deposit_borrow_ratio'].clip(0, 10) * 30) +  # 0-300 from ratio
        (ml_wallets['time_since_first_tx'].clip(0, 365) / 365 * 200) +  # 0-200 from age
        (ml_wallets['n_repays'] * 2)  # 0-200 from repay count (assuming max 100 repays)
    ).clip(300, 800)  # Keep in reasonable range

    # Train model
    X = ml_wallets[features]
    y = ml_wallets['target']

    model = LGBMRegressor(
        n_estimators=100,
        max_depth=5,
        random_state=42
    )

    model.fit(X, y)

    # Predict scores
    ml_wallets['ml_score'] = model.predict(X)
    wallets.loc[ml_wallets.index, 'ml_score'] = ml_wallets['ml_score']

    return wallets

In [42]:
# Step 9: Combine Scores
def combine_scores(wallets):
    print("Combining scores...")

    wallets['final_score'] = np.where(
        wallets['rule_based_score'].notna(),
        wallets['rule_based_score'],
        wallets['ml_score']
    )

    # Fill any remaining NAs with median
    median_score = wallets['final_score'].median()
    wallets['final_score'] = wallets['final_score'].fillna(median_score)

    # Scale to 0-1000 range
    wallets['final_score'] = wallets['final_score'].clip(0, 1000)
    wallets['final_score'] = wallets['final_score'].round().astype(int)

    return wallets

In [43]:
# Step 10: Analysis and Visualization
def analyze_results(wallets):
    print("Analyzing results...")

    # Score distribution
    plt.figure(figsize=(10, 6))
    bins = range(0, 1100, 100)
    plt.hist(wallets['final_score'], bins=bins, edgecolor='black')
    plt.title('Wallet Credit Score Distribution', fontsize=14)
    plt.xlabel('Credit Score', fontsize=12)
    plt.ylabel('Number of Wallets', fontsize=12)
    plt.grid(axis='y', alpha=0.75)
    plt.savefig('score_distribution.png')
    plt.close()

    # Score statistics
    analysis = {
        'n_wallets': len(wallets),
        'avg_score': wallets['final_score'].mean(),
        'median_score': wallets['final_score'].median(),
        'min_score': wallets['final_score'].min(),
        'max_score': wallets['final_score'].max(),
        'score_distribution': pd.cut(wallets['final_score'], bins=bins).value_counts().sort_index().to_dict()
    }

    # Save results
    wallets[['userWallet', 'final_score']].to_csv('wallet_scores.csv', index=False)

    # Create analysis markdown
    with open('analysis.md', 'w') as f:
        f.write("# Aave V2 Wallet Credit Score Analysis\n\n")
        f.write("## Summary Statistics\n")
        f.write(f"- Total wallets scored: {analysis['n_wallets']:,}\n")
        f.write(f"- Average score: {analysis['avg_score']:.1f}\n")
        f.write(f"- Median score: {analysis['median_score']:.1f}\n")
        f.write(f"- Minimum score: {analysis['min_score']}\n")
        f.write(f"- Maximum score: {analysis['max_score']}\n\n")

        f.write("## Score Distribution\n")
        f.write("![Score Distribution](score_distribution.png)\n\n")
        f.write("| Score Range | Number of Wallets |\n")
        f.write("|-------------|-------------------|\n")
        for bin_range, count in analysis['score_distribution'].items():
            f.write(f"| {bin_range} | {count:,} |\n")

        f.write("\n## Wallet Behavior Analysis\n")
        f.write("### High-Score Wallets (700-1000)\n")
        f.write("- Consistent deposit/borrow patterns\n")
        f.write("- Long protocol tenure\n")
        f.write("- Healthy collateralization ratios\n")
        f.write("- No liquidations\n")
        f.write("- Higher scores for wallets with good deposit/borrow ratios and long history\n\n")


        f.write("### Low-Score Wallets (0-300)\n")
        f.write("- Frequent liquidations\n")
        f.write("- Bot-like transaction patterns\n")
        f.write("- New accounts with risky behavior\n")
        f.write("- Borrowing without deposits\n\n")

        f.write("### Middle-Range Wallets (300-700)\n")
        f.write("- Mixed behavior patterns\n")
        f.write("- Some risk factors but not extreme\n")
        f.write("- Potential for score improvement\n")

    return analysis

In [44]:
# Step 11: Main Execution
def main():
    # Load data
    try:
        df = load_data(file_path)
    except Exception as e:
        print(f"Error loading data: {e}")
        return

    # Engineer features
    wallets = engineer_features(df)

    # Rule-based scoring
    wallets = rule_based_scoring(wallets)

    # ML scoring
    wallets = ml_scoring(wallets)

    # Combine scores
    wallets = combine_scores(wallets)

    # Analyze results
    analysis = analyze_results(wallets)

    # Print summary
    print("\n=== Analysis Summary ===")
    print(f"Total wallets scored: {analysis['n_wallets']:,}")
    print(f"Average score: {analysis['avg_score']:.1f}")
    print(f"Median score: {analysis['median_score']:.1f}")
    print(f"Score distribution:")
    for bin_range, count in analysis['score_distribution'].items():
        print(f"  {bin_range}: {count:,} wallets")

    print("\nResults saved to:")
    print("- wallet_scores.csv (all wallet addresses with scores)")
    print("- score_distribution.png (histogram of scores)")
    print("- analysis.md (detailed analysis)")

# Run the main function
if __name__ == '__main__':
    main()

Loading data from user-wallet-transactions.json...
Creating DataFrame...
Loaded 100000 transactions
Engineering features...
Engineered features for 3497 wallets
Applying rule-based scoring...
Applying machine learning scoring...
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000503 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1526
[LightGBM] [Info] Number of data points in the train set: 3305, number of used features: 9
[LightGBM] [Info] Start training from score 451.074749
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive g